# configuration

In [ ]:
from scheduler import *

s = scheduler(
            pioneerKeyPath = '/home/foivos/.ssh/monroeResults', # <-- change this to point to your own pioneer key
            pioneerUser = 'foivos',  # <-- change this to point to your own pioneer user
            userId = 245, # <-- change this to your own user ID
            pendingFilesClientDir = "../pendingFilesClient/", # directory where we save the results of the experiments
            processedFilesClientDir = "../processedFilesClient/",
            supportDir = './', # directory where we store the intermediate scheduling files.
            PemKeyDir = '/home/foivos/foivos.key.pem', # pem key directory
            PemCerDir = '/home/foivos/foivos.crt.pem' # pem certificate directory
            )

# Retrieve a list of the scheduled experiments (past and future) and download related results of a user scheduled after a date.

In [ ]:
startDate = "2023-05-30 00:00:00"

scheduledExepriments = s.getUserExperimentsDate(startDate)
expIds = getExperimentIds(scheduledExepriments)
epxerimentElements = s.getSpecificExperimentElements(expIds)
for experiment, elementList in epxerimentElements.items():
    for element in elementList:
        print(element)
        try:
            s.downloadExperimentalElementResults(element)
        except:
            print("Not able to download")
            continue

# Download experiments

In [ ]:
s.downloadExperimentalElementResults(2985817)

# submit experiment to a list of nodes

In [ ]:
# check in scheduler.py for predifined node lists.
nodeList = [4125]
# nodeList = new_5G_nodes

# start at a specific UTC time
# start = getTimestamp('2021-04-08 23:00:00')
# alternatively start 10 minutes from now
# you need to schedule experiments at least 5 minutes from now to allow the scheduler to
# communicate them to the nodes in time.
start = int(time.time() + 600)

duration = 600
nodecount = 1
experimentName = "simple ping programmatically"
containerURL = "docker.io/crnaeng/simpleping"
optionsList = [    
    {
        "targets": ["www.vg.no"],
         "numberOfPings": 5,
         "Operator": ["Telenor", "Telia"]
    }
]

# j: how many times to run an experiment per nonde
# i: some time spacing between nodes to make sure that they do not send requests to the same servers at the same time
# x: The different configurations, such as operators / interfaces
for j in range(1):
    for i in range(len(nodeList)):
        for x in range(len(optionsList)):
            initiallyRequestedStartTime = start + (3600 * j) + (i * 300) + (600 * x)
            nodeAvailability = s.checkAvailability(nodeList[i], initiallyRequestedStartTime, duration, nodecount)
            nonConflictingStartTime = nodeAvailability[0]["start"]
            print(f"Initially requested start time: UTC timestamp {initiallyRequestedStartTime}, UTC human readable: {humanDate(initiallyRequestedStartTime)}")
            print(f"nonConflictingStartTime: {nonConflictingStartTime}, UTC human readable: {humanDate(nonConflictingStartTime)}")
            s.submitExperiment(nodeList[i], nonConflictingStartTime, duration, nodecount, f"{experimentName} on node {str(nodeList[i])}", containerURL, optionsList[x])